<a href="https://colab.research.google.com/github/ThePlator/Finetune/blob/main/finetune_ipygggnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
pip install unsloth transformers trl


In [34]:
import torch
from unsloth import FastLanguageModel
from  datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

In [35]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name= "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [36]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], # <--- Fixed spelling here
    # ... add other parameters if needed (lora_alpha, etc.)
)

In [37]:
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

In [38]:
dataset = load_dataset("mlabonne/FineTome-100k", split="train")

In [39]:
dataset = standardize_sharegpt(dataset)

In [40]:
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [41]:
dataset[0]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [42]:
dataset = dataset.map(
    lambda examples:{
        "text":[
            tokenizer.apply_chat_template(convo,tokenize=False)
            for convo in examples["conversations"]
        ]
    },
    batched = True
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [43]:
dataset[0]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [44]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset= dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        logging_steps=1,
        output_dir="output",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [45]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: contact-sameer04 (contact-sameer04-purnea-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.439900
2,1.846100
3,1.380300
4,1.414000
5,1.350800
6,1.445900
7,0.946300
8,1.507300
9,1.239200
10,1.254100


train/epoch,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇██
train/grad_norm,▅▆█▇▅▆▄▄▃▃▂▂▂▁▂▁▃▂▃▂▁▁▃▂▃▂▂▁▁▁▂▁▂▂▂▂▃▃▃▂
train/learning_rate,▂▄▇███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁
train/loss,█▅▅▅▅▆▄▄▂▂▃▄▃▃▂▂▄▂▂▂▄▂▄▃▃▃▃▂▃▁▂▁▂▂▂▂▂▂▂▂
total_flos,5495762487078912.0
train/epoch,0.0048
train/global_step,60
train/grad_norm,0.18603
train/learning_rate,0.0
train/loss,0.8743


TrainOutput(global_step=60, training_loss=1.0265153706073762, metrics={'train_runtime': 659.777, 'train_samples_per_second': 0.728, 'train_steps_per_second': 0.091, 'total_flos': 5495762487078912.0, 'train_loss': 1.0265153706073762, 'epoch': 0.0048})

In [46]:
model.save_pretrained("finetunned_model")

In [48]:
inference_model, inference_tokenizer = FastLanguageModel.from_pretrained(
    model_name="./finetunned_model",
    max_seq_length=2048,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [49]:
text_prompts = [
    "what are the key priciples of investment?"
]

for prompt in text_prompts:
    formatted_prompt = inference_tokenizer.apply_chat_template([
        {
            "role": "user",
            "content": prompt
        }
    ], tokenize=False)

    model_inputs = inference_tokenizer(
        formatted_prompt,
        return_tensors="pt"
    ).to("cuda")

    generated_ids = inference_model.generate(
        **model_inputs,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        pad_token_id=inference_tokenizer.pad_token_id
    )

    response = inference_tokenizer.batch_decode(
        generated_ids,
        skip_special_tokens=True
    )[0]

    print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 27 Nov 2025

user

what are the key priciples of investment?assistant

The key principles of investment are:

1. Diversification: Investing in a variety of assets to reduce risk and increase potential returns.
2. Long-term perspective: Investing for the long-term, rather than trying to make quick profits.
3. Risk management: Understanding and managing risk to minimize potential losses.
4. Dollar-cost averaging: Investing a fixed amount of money at regular intervals, regardless of market conditions.
5. Compounding: Reinvesting earnings to take advantage of the power of compounding.
6. Low-cost investing: Choosing low-cost index funds or ETFs to minimize fees and maximize returns.
7. Tax efficiency: Considering the tax implications of investments and aiming to minimize tax liabilities.
8. Active management: Actively managing investment portfolios to take advantage of market opportunities and minimize risks.
9. Estate planning: Con